In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))



df = pd.read_csv(os.path.join(os.getcwd(), "../../data/total_4am.csv"), index_col=0)


In [2]:
import pandas as pd
from performance import get_performance, draw_graph

def various_ma(interval_list, df):

    original_df = df.copy()  # Make a copy of the original DataFrame

    for interval in interval_list:
        df = original_df.copy()  # Create a fresh copy for each interval

        df[f"{interval}_ma"] = df['close'].rolling(window=interval).mean()

        # Implement RSI strategy for long positions only
        df['signal'] = 0  # Default to no position
        for i in range(interval, len(df)):
            # Buy condition
            if df.loc[i, 'close'] >= df.loc[i, f'{interval}_ma'] and df.loc[i-1, 'close'] < df.loc[i-1, f'{interval}_ma']:
                df.loc[i, 'signal'] = 1
            # Sell condition
            elif df.loc[i, 'close'] < df.loc[i, f'{interval}_ma'] and df.loc[i-1, 'close'] >= df.loc[i-1, f'{interval}_ma']:
                df.loc[i, 'signal'] = -1

        # Manage positions with stop loss, take profit, and sell signal
        df['position'] = 0
        df['highest_price'] = np.nan
        df['exit_price'] = np.nan
        holding_position = False

        for i in range(1, len(df)):
            if df['signal'].iloc[i] == 1 and not holding_position:
                # Enter position
                df.loc[i, 'position'] = 1
                df.loc[i, 'highest_price'] = df.loc[i, 'close']
                holding_position = True
            elif holding_position:
                # Calculate percentage change since entry
                # df['highest_price'].iloc[i] = max(df['highest_price'].iloc[i-1], df['close'].iloc[i])
                df.loc[i, 'highest_price'] = max(df.loc[i-1, 'highest_price'], df.loc[i-1, 'close'])
                highest_price = df['highest_price'].iloc[i]
                current_price = df['close'].iloc[i]
                percent_change = (current_price - highest_price) / highest_price * 100

                if df['signal'].iloc[i] == -1:  # Sell signal condition
                    # print(f"cond1 on{i}")
                    df.loc[i, 'position'] = 0
                    df.loc[i, 'exit_price'] = current_price
                    holding_position = False
                elif percent_change <= -5:  # Stop loss condition
                    # print(f"cond2 on{i}")
                    df.loc[i, 'position'] = 0
                    df.loc[i, 'exit_price'] = current_price
                    holding_position = False
                # elif percent_change >= 30:  # Take profit condition
                #     print(f"cond3 on{i}")
                #     df['position'].iloc[i] = 0
                #     df['exit_price'].iloc[i] = current_price
                #     holding_position = False
                else:
                    # Continue holding the position if no sell conditions are met
                    df.loc[i, 'position'] = df.loc[i-1, 'position']

            else:
                # No signal and no position
                # df['position'].iloc[i] = df['position'].iloc[i-1]
                df.loc[i, 'position'] = df.loc[i-1, 'position']
                
        # Calculate the strategy returns (only when in a long position)
        df['strategy_returns'] = df['position'].shift(1) * df['close'].pct_change()

        df['strategy_returns2'] = df['strategy_returns']
        
        for i in range(1, len(df)):
            buy_price = df.loc[i-1, "close"]
            buy_price_copy = buy_price
            sell_price = df.loc[i, "close"]
            sell_price_copy = sell_price
            if df.loc[i-1, 'position'] == 1 and df.loc[i-1, 'signal'] == 1:
                # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'])/(df.loc[i-1, 'position'] * 1.002) -1
                buy_price = df.loc[i-1, "close"] * 1.002
            if df.loc[i, 'position'] == 0 and df.loc[i-1, 'position'] != 0:
                # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'] * 0.998)/(df.loc[i-1, 'position']) -1
                sell_price = df.loc[i, "close"] * 0.998
            
            if buy_price == buy_price_copy and sell_price == sell_price_copy:
                continue


            df.loc[i, "strategy_returns2"] = sell_price/buy_price - 1

        df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

        df["cumulative_returns2"] = (1 + df['strategy_returns2']).cumprod()

        # Calculate the benchmark cumulative returns (buy and hold strategy)
        df['benchmark_returns'] = (1 + df['close'].pct_change()).cumprod()

        # Save the results to a CSV file
        df.to_csv(f"../../results/ma/ma{(interval)}_cut_highest.csv")

        # Get performance metrics and plot the results
        excel_path = "../../results/results.xlsx"
        get_performance(df=df, title=f"ma_{interval}_cut_highest5%", add_to_excel=True, file_path=excel_path)
        # draw_graph(df=df)

        
    df = original_df.copy()  # Create a fresh copy for each interval


In [3]:
interval_list = [5, 10, 20, 50, 90, 120, 200]
various_ma(interval_list=interval_list, df=df)

======Investment Summary======
Strategy        : ma_5_cut_highest5%
total_return    : 284.08
cagr            : 48.99
mdd             : 44.87
total_return_w_fee : 112.48
cagr_w_fee      : 25.02
mdd_w_fee       : 55.89
investing_days  : 1232
======Investment Summary======
Strategy        : ma_10_cut_highest5%
total_return    : 211.81
cagr            : 41.04
mdd             : 33.45
total_return_w_fee : 115.81
cagr_w_fee      : 26.19
mdd_w_fee       : 42.26
investing_days  : 1207
======Investment Summary======
Strategy        : ma_20_cut_highest5%
total_return    : 176.38
cagr            : 36.1
mdd             : 30.66
total_return_w_fee : 117.41
cagr_w_fee      : 26.55
mdd_w_fee       : 38.15
investing_days  : 1204
======Investment Summary======
Strategy        : ma_50_cut_highest5%
total_return    : 270.26
cagr            : 53.15
mdd             : 29.42
total_return_w_fee : 235.02
cagr_w_fee      : 48.24
mdd_w_fee       : 33.13
investing_days  : 1121
======Investment Summary======
Strateg